In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pybaseball import statcast

#df vizi settings
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize']= [10,5]
plt.rcParams['font.family'] = 'serif'
plt.style.use('dark_background')


In [20]:
df=statcast(start_dt="2023-8-29", end_dt="2023-10-30")

This is a large query, it may take a moment to complete


c:\Users\Brett\miniconda3\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 63/63 [00:20<00:00,  3.04it/s]


In [21]:
df.describe()

,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,...,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp
count,147599,147595.0,147595.0,147595.0,147599.0,147599.0,0.0,0.0,0.0,0.0,...,147599.0,147599.0,147599.0,147599.0,147599.0,147599.0,147599.0,147538.0,147599.0,147584.0
mean,2023-09-16 18:16:47.860486912,89.171184,-0.856551,5.762354,634498.924505,629286.539529,<NA>,<NA>,<NA>,<NA>,...,2.472002,2.285693,2.337706,2.487815,2.16639,2.316499,2.337706,177.617393,-0.000029,0.000032
min,2023-08-29 00:00:00,36.1,-4.63,0.9,408234.0,425794.0,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.667,-1.13
25%,2023-09-07 00:00:00,84.8,-2.15,5.49,606466.0,605400.0,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,0.0,-0.065
50%,2023-09-16 00:00:00,90.1,-1.54,5.81,663368.0,656464.0,<NA>,<NA>,<NA>,<NA>,...,2.0,2.0,1.0,2.0,1.0,2.0,1.0,203.0,0.0,-0.017
75%,2023-09-26 00:00:00,94.1,0.74,6.09,670541.0,669459.0,<NA>,<NA>,<NA>,<NA>,...,4.0,4.0,4.0,4.0,3.0,4.0,4.0,224.0,0.0,0.036
max,2023-10-30 00:00:00,103.7,4.62,7.37,807799.0,701643.0,<NA>,<NA>,<NA>,<NA>,...,20.0,20.0,20.0,20.0,16.0,20.0,20.0,360.0,0.858,3.581
std,NaN,6.105857,1.838311,0.527209,56365.581939,57768.921958,<NA>,<NA>,<NA>,<NA>,...,2.761965,2.585718,2.732926,2.767848,2.55173,2.599011,2.732926,73.157054,0.028327,0.244378


In [14]:
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [22]:
#build an XGBoost Model that predicts if a given pitch is going to a HR or not
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load and preprocess your dfset, selecting the specified features and 'events' as the target
# Adjust these lines based on your dfset structure
features = df[['release_speed', 'release_pos_x', 'release_pos_z', 'hit_location', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y', 'launch_speed_angle', 'at_bat_number']]
target = (df['events'] == 'home_run')

# Split the df into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Create an instance of the XGBoost classifier
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.9996838301716351
Confusion Matrix:
[[43872    11]
 [    3   394]]
Classification Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     43883
        True       0.97      0.99      0.98       397

    accuracy                           1.00     44280
   macro avg       0.99      1.00      0.99     44280
weighted avg       1.00      1.00      1.00     44280



In [25]:
unknown_df=statcast(start_dt="2023-3-29", end_dt="2023-7-30")

This is a large query, it may take a moment to complete


c:\Users\Brett\miniconda3\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 124/124 [01:42<00:00,  1.21it/s]


In [26]:
# Make predictions on the unknown df and evaluate the accuracy of predicting the 'events' column of home_run or not
# Adjust these lines based on your dfset structure
unknown_features = unknown_df[['release_speed', 'release_pos_x', 'release_pos_z', 'hit_location', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y', 'launch_speed_angle', 'at_bat_number']]
unknown_target = (unknown_df['events'] == 'home_run')

unknown_pred = model.predict(unknown_features)

unknown_accuracy = accuracy_score(unknown_target, unknown_pred)
unknown_confusion = confusion_matrix(unknown_target, unknown_pred)
unknown_classification_rep = classification_report(unknown_target, unknown_pred)

print(f"Accuracy: {unknown_accuracy}")
print("Confusion Matrix:")
print(unknown_confusion)
print("Classification Report:")
print(unknown_classification_rep)

Accuracy: 0.9996943954693595
Confusion Matrix:
[[464078    102]
 [    41   3704]]
Classification Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00    464180
        True       0.97      0.99      0.98      3745

    accuracy                           1.00    467925
   macro avg       0.99      0.99      0.99    467925
weighted avg       1.00      1.00      1.00    467925



In [31]:
import shap

# Assuming you already have your XGBoost model trained and loaded as 'model'
# You can use the 'model' variable from your previous code

# Load and preprocess your dataset again, selecting the specified features
# Adjust these lines based on your dataset structure
features = unknown_df[['pitch_type','release_speed', 'release_pos_x', 'release_pos_z', 'hit_location', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y', 'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name']]

# Create an explainer using the trained XGBoost model
explainer = shap.Explainer(model)

# Calculate SHAP values for a single prediction (you can change the index)
shap_values = explainer.shap_values(features.iloc[4])

# Display the SHAP values summary plot for that prediction
shap.summary_plot(shap_values, features, feature_names=features.columns)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`. 

In [6]:
#visalize the win expectancy over each pitch of the game
import matplotlib.pyplot as plt